In [2]:
!pip install transformers datasets

     |████████████████████████████████| 1.8MB 9.1MB/s 
     |████████████████████████████████| 163kB 37.3MB/s 
     |████████████████████████████████| 2.9MB 35.9MB/s 
     |████████████████████████████████| 890kB 40.7MB/s 
     |████████████████████████████████| 20.7MB 77.3MB/s 
     |████████████████████████████████| 245kB 56.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=38ae2050a76ca2e6cb021a92f3a9e8e6f97afdf1b1d043fce8097cb9253fc8f0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [3]:
import os
import sys
import pandas
import pickle
import json
import torch
import numpy
import warnings
warnings.filterwarnings('ignore') #Some operations warn inside a loop

device = torch.device("cuda:0")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name()

'Tesla T4'

In [4]:
print(device)

cuda:0


In [5]:
#Grant access to your local g-drive
from google.colab import drive
drive.mount('/content/drive/')
path = '/content/drive/My Drive/Colab Notebooks/aips/ch13/'

Mounted at /content/drive/


In [7]:
import datasets
datadict = datasets.load_from_disk(path+'data/outdoors_questionanswer_datadict')

In [8]:
import transformers
tokenizer = transformers.RobertaTokenizerFast.from_pretrained('roberta-base')
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [9]:
#This method adopted from the following example notebook, Apache 2.0 licensed:
#https://github.com/huggingface/notebooks/blob/master/examples/question_answering.ipynb
def tokenize_dataset(examples):

    maximum_tokens = 384 # This will be the number of tokens in BOTH the question and context
    document_overlap = 128 # Sometimes we need to split the context into smaller chunks, so we will overlap with this window
    pad_on_right = tokenizer.padding_side == "right"
    
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=maximum_tokens,
        stride=document_overlap,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )
    
    print(tokenized_examples[0])

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [11]:
"""
To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the map method of our dataset object we created earlier. 
This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command. 
Since our preprocessing changes the number of samples, we need to remove the old columns when applying it.
 --Huggingface
"""
tokenized_datasets = datadict.map(tokenize_dataset, batched=True, remove_columns=datadict["train"].column_names)

Encoding(num_tokens=384, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


Encoding(num_tokens=384, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

Encoding(num_tokens=384, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


Encoding(num_tokens=384, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])



In [12]:
tokenized_datasets.save_to_disk(path+'data/outdoors_questionanswer_datadict_tokenized')

In [13]:
from transformers import RobertaForQuestionAnswering, TrainingArguments, Trainer, default_data_collator
import torch

model = RobertaForQuestionAnswering.from_pretrained('deepset/roberta-base-squad2')

training_args = TrainingArguments(
    output_dir=path+'data/questionanswering/results',     # output directory
    evaluation_strategy = "epoch",                        # evaluate loss per epoch
    num_train_epochs=3,                                   # total # of training epochs
    per_device_train_batch_size=16,                       # batch size per device during training
    per_device_eval_batch_size=64,                        # batch size for evaluation
    warmup_steps=500,                                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                                    # strength of weight decay
    logging_dir=path+'data/questionanswering/logs'        # directory for storing logs
)

trainer = Trainer(
    model=model,                                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                                   # training arguments, defined above
    data_collator=default_data_collator,                  
    tokenizer=tokenizer,                                  
    train_dataset=tokenized_datasets['train'],            # training dataset
    eval_dataset=tokenized_datasets['test']               # evaluation dataset
)

In [14]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,3.249251,0.561000,48.132000
2,No log,2.854391,0.566600,47.656000
3,No log,2.511550,0.569200,47.434000


TrainOutput(global_step=24, training_loss=2.2902100880940757, metrics={'train_runtime': 24.9061, 'train_samples_per_second': 0.964, 'total_flos': 96895134586368, 'epoch': 3.0})

In [15]:
trainer.save_model(path+"data/test-outdoors-trained")

In [16]:
#TODO: Needs a holdout dataset
trainer.evaluate()

{'epoch': 3.0,
 'eval_loss': 2.511549949645996,
 'eval_runtime': 0.6422,
 'eval_samples_per_second': 42.04}

In [ ]:
import tqdm
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
my_model = path+"data/test-outdoors-trained"
nlp2 = pipeline('question-answering', model=my_model, tokenizer=my_model,device=0)
def answer_questions(answers,n=10):
    guesses = []
    for answer in tqdm.tqdm(answers[0:n]):
        result = nlp2(answer)
        answer["result"] = result
        #print(answer['question'])
        #print(result)
        guesses.append(answer)
    return guesses

Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/My Drive/Colab Notebooks/aips/ch13/data/test-outdoors-trained and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
contexts = json.load(open(path + 'data/outdoors_question_answering_contexts.json','r'))
guesses = answer_questions(contexts[500:600],n=100)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ResourceWarning: unclosed file <_io.TextIOWrapper name='/content/drive/My Drive/Colab Notebooks/aips/ch13/data/outdoors_question_answering_contexts.json' mode='r' encoding='UTF-8'>
  """Entry point for launching an IPython kernel.
100%|██████████| 100/100 [00:03<00:00, 27.51it/s]


In [ ]:
guesses

[{'context': "It's the Great Peacock Moth Caterpillar . See Wikipedia for other images, including a painting of the moth by Van Gogh. Saturnia pyri, the giant peacock moth, also called the great peacock moth, giant emperor moth, or Viennese emperor, is a Saturniid moth which is native to Europe. It is the largest European moth, with a wingspan reaching 15-20 cm. In French it is called Grand paon de nuit. The picture on the left is from Caterpillars, Romania . The picture on the right shows the caterpillar and the moth. It's from Worldwide Butterflies . There's an excellent picture of the entire lifecyle of the moth here . It shows the cocoons and moths in various stages while they change color and shape as they mature. There's a charge to download it, so I didn't do that, but I highly recommend checking it out.",
  'question': 'What will this bright green caterpillar with small turquoise spots turn into?',
  'result': {'answer': 'Great Peacock Moth Caterpillar',
   'end': 39,
   'score